In [1]:
# Import necessary libraries
from pyspark.sql import SparkSession

# Create a local SparkSession
# This is the entry point to all Spark functionality.
# .master("local[*]") tells Spark to use all available cores on your local machine.
# .appName(...) gives your application a name.
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("NYCTransitDataAnalysis") \
    .getOrCreate()

# Print the SparkSession object to confirm it's created
print(spark)